# Model 1 Logistic Regression

In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [3]:
import pandas as pd

## Read the CSV and Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("Resources/exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


## Select your features (columns)

In [5]:
df.info

<bound method DataFrame.info of      koi_disposition  koi_fpflag_nt  koi_fpflag_ss  koi_fpflag_co  \
0          CONFIRMED              0              0              0   
1     FALSE POSITIVE              0              1              0   
2     FALSE POSITIVE              0              1              0   
3          CONFIRMED              0              0              0   
4          CONFIRMED              0              0              0   
...              ...            ...            ...            ...   
6986  FALSE POSITIVE              0              0              0   
6987  FALSE POSITIVE              0              1              1   
6988       CANDIDATE              0              0              0   
6989  FALSE POSITIVE              0              0              1   
6990  FALSE POSITIVE              0              0              1   

      koi_fpflag_ec  koi_period  koi_period_err1  koi_period_err2  \
0                 0   54.418383     2.479000e-04    -2.479000e-04   
1

In [6]:
# Set features. This will also be used as your x values.
# Assign X (data) and y (target)
X = df.drop('koi_disposition', axis=1)
X = X.drop('koi_tce_plnt_num', axis=1)

y = df['koi_disposition']

print(X.shape, y.shape)

(6991, 39) (6991,)


## Create a Train Test Split

Use `koi_disposition` for the y values

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [8]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
6122,0,0,0,0,6.768901,7.380000e-05,-7.380000e-05,133.077240,0.008440,-0.008440,...,-171,4.327,0.153,-0.187,1.125,0.310,-0.207,294.40472,39.351681,14.725
6370,0,1,0,1,0.733726,6.060000e-06,-6.060000e-06,132.020050,0.007950,-0.007950,...,-175,4.578,0.033,-0.187,0.797,0.211,-0.056,284.50391,42.463860,15.770
2879,1,0,0,0,7.652707,6.540000e-05,-6.540000e-05,134.460380,0.006190,-0.006190,...,-189,4.481,0.050,-0.200,0.963,0.290,-0.097,295.50211,38.983540,13.099
107,0,0,0,0,7.953547,1.910000e-05,-1.910000e-05,174.662240,0.001820,-0.001820,...,-85,4.536,0.056,-0.016,0.779,0.023,-0.049,291.15878,40.750271,15.660
29,0,0,0,0,4.959319,5.150000e-07,-5.150000e-07,172.258529,0.000083,-0.000083,...,-77,4.359,0.110,-0.110,1.082,0.173,-0.130,292.16705,48.727589,15.263


## Pre-processing

Scale the data using the MinMaxScaler and encode y 

In [9]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# Scale your data
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

## Create and Fit the Model

In [10]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=400)
model.fit(X_train_scaled, encoded_y_train)

LogisticRegression(max_iter=400)

## Train the Model

In [11]:
print(f"Training Data Score: {model.score(X_train_scaled, encoded_y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, encoded_y_test)}")

Training Data Score: 0.8504672897196262
Testing Data Score: 0.8426773455377574


In [12]:
from sklearn.metrics import classification_report

predictions = model.predict(X_test_scaled)
print(classification_report(encoded_y_test, predictions, 
                            target_names = ["confirmed", "false positive", "candidate"]))
print(f"First 10 Predictions:   {predictions[:10].tolist()}")
print(f"First 10 Actual labels: {encoded_y_test[:10].tolist()}")

                precision    recall  f1-score   support

     confirmed       0.70      0.63      0.66       411
false positive       0.71      0.75      0.73       484
     candidate       0.98      1.00      0.99       853

      accuracy                           0.84      1748
     macro avg       0.80      0.79      0.79      1748
  weighted avg       0.84      0.84      0.84      1748

First 10 Predictions:   [2, 0, 2, 2, 2, 1, 1, 0, 0, 2]
First 10 Actual labels: [2, 0, 2, 2, 2, 1, 0, 0, 0, 2]


## Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [13]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [10, 50, 100],
              'max_iter': [800, 1000, 1500]}
grid = GridSearchCV(model, param_grid, verbose=3)

In [14]:
# Train the model with GridSearch
grid.fit(X_train_scaled, encoded_y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5] END ................C=10, max_iter=800;, score=0.887 total time=   2.1s
[CV 2/5] END ................C=10, max_iter=800;, score=0.866 total time=   1.6s
[CV 3/5] END ................C=10, max_iter=800;, score=0.877 total time=   1.8s
[CV 4/5] END ................C=10, max_iter=800;, score=0.853 total time=   2.3s
[CV 5/5] END ................C=10, max_iter=800;, score=0.866 total time=   1.7s
[CV 1/5] END ...............C=10, max_iter=1000;, score=0.887 total time=   2.3s
[CV 2/5] END ...............C=10, max_iter=1000;, score=0.866 total time=   1.6s
[CV 3/5] END ...............C=10, max_iter=1000;, score=0.877 total time=   1.8s
[CV 4/5] END ...............C=10, max_iter=1000;, score=0.853 total time=   2.3s
[CV 5/5] END ...............C=10, max_iter=1000;, score=0.866 total time=   1.6s
[CV 1/5] END ...............C=10, max_iter=1500;, score=0.887 total time=   2.0s
[CV 2/5] END ...............C=10, max_iter=1500;,

C:\Users\robin\anaconda3\envs\PythonAdv\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[CV 1/5] END ................C=50, max_iter=800;, score=0.894 total time=   2.8s


C:\Users\robin\anaconda3\envs\PythonAdv\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[CV 2/5] END ................C=50, max_iter=800;, score=0.879 total time=   2.8s


...

[CV 4/5] END ..............C=100, max_iter=1500;, score=0.867 total time=   8.8s


C:\Users\robin\anaconda3\envs\PythonAdv\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[CV 5/5] END ..............C=100, max_iter=1500;, score=0.870 total time=   8.2s


C:\Users\robin\anaconda3\envs\PythonAdv\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


GridSearchCV(estimator=LogisticRegression(max_iter=400),
             param_grid={'C': [10, 50, 100], 'max_iter': [800, 1000, 1500]},
             verbose=3)

In [ ]:
print(grid2.best_params_)
print(grid2.best_score_)

# Save the Model

In [ ]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'your_name.sav'
joblib.dump(your_model, filename)